In [22]:
# To Do: Create global config object for things that are not tied to a
#        particular dataset configuration but may still be changeable, e.g.
#        user messages related to input validation. Or maybe not this is an
#        interactive notebook not some compiled software.

import ipywidgets as widgets
import json
import datetime
import warnings

from IPython.display import display

with open('./default/default_employees.json') as default_configuration_json:
    config = json.load(default_configuration_json)

In [2]:
# For procedurally generated widgets assign them an ID so that we can safely go
# back and remove them later as required.
current_widget_id = 0
    
def obtain_new_widget_id():
    global current_widget_id
    current_widget_id += 1
    return current_widget_id

In [3]:
# Standard Properties
l3_box_layout = widgets.Layout(
      display         = 'flex'
    , flex_flow       = 'column'
    , align_content   = 'center'
    , justify_content = 'center'
)

l3_text_layout = widgets.Layout(
    display           = 'flex'
  , justify_content   = 'center'
  , width             = '25%'
)

l3_input_layout = widgets.Layout(
    width             = 'max_content'
)

l3_input_style = {
    'description_width': '50%'
}

In [4]:
# Global Object to hold all widgets based on where they will be rendered. While
# ipywidgets supports re-rendering the same widget in multiple/cells views we
# will not use this methodology in this notebook.
ws = {
      'l1_tab': widgets.Tab()

# Level 2: Overview ----------------------------------------------------------#
    , 'l2_overview':                           widgets.Box()
    , 'l3_dataset_details':                    widgets.Box(layout = l3_box_layout)
    , 'l3_dataset_details_load_configuration': widgets.Button(description = 'Load Configuration')
    , 'l3_dataset_details_save_configuration': widgets.Button(description = 'Save Configuration')

    , 'l3_dataset_details_configuration_name': widgets.Text(
          value       = ''
        , description = 'Configuration Name'
        , disabled    = False
        , layout      = l3_input_layout
        , style       = l3_input_style
    )

    , 'l3_dataset_details_dataset_configuration_last_modified': widgets.Text(
          value       = ''
        , description = 'Last Modified'
        , disabled    = True
        , layout      = l3_input_layout
        , style       = l3_input_style
    )

    , 'l3_dataset_details_dataset_name': widgets.Text(
          value       = ''
        , description = 'Dataset Name'
        , disabled    = True
        , layout      = l3_input_layout
        , style       = l3_input_style
    )
# ----------------------------------------------------------------------------#

# Level 2: Employee Properties ----------------------------------------------#
    , 'l2_employee_properties': widgets.Accordion()
    
    , 'l3_general_organisation_size': widgets.BoundedIntText(
          value       = 100
        , min         = 100
        , max         = 100000
        , description = 'Organisation Size'
        , disabled    = False
        , layout      = l3_input_layout
        , style       = l3_input_style
    )
    
    , 'l3_employment_type_mix':               widgets.Box(layout = l3_box_layout)
    , 'l3_employment_type_mix_header':        widgets.Box(layout = l3_box_layout)
    , 'l3_employment_type_mix_user_message':  widgets.Label(value = '')
    , 'l3_employment_type_mix_add_type':      widgets.Button(description = 'Add Type')    
    , 'l3_employment_type_mix_column_titles': widgets.HBox()
    , 'l3_employment_type_mix_total':         widgets.IntText(
              description = 'Total'
            , value       = 0
            , disabled    = True
          # , layout      = l3_input_layout
          # , style       = l3_input_style
        )
    , 'l3_employment_type_mix_body':          widgets.Box(layout = l3_box_layout)
    
    , 'l3_position_level_modifiers':               widgets.Box(layout = l3_box_layout)
    , 'l3_position_level_modifiers_add_modifier':  widgets.Button(description = 'Add Modifier')
    , 'l3_position_level_modifiers_column_titles': widgets.HBox()
    , 'l3_position_level_modifiers_body':          widgets.Box(layout = l3_box_layout)
    

# ----------------------------------------------------------------------------#
}

In [18]:
# Level 3: Overview / Dataset Details
def l3_overview_save_configuration():
    ws['l3_dataset_details_dataset_configuration_last_modified'].value = str(datetime.datetime.now(datetime.timezone.utc))
    
    output = {
    "overview": {
        "dataset_details": {
            "configuration_name": ws['l3_dataset_details_configuration_name'].value,
            "last_modified": ws['l3_dataset_details_dataset_configuration_last_modified'].value,
            "dataset_name": "employees"
        }
    },
    "employee_properties": {
        "organisation_size": {
            "description": "Organisation Size",
            "value": ws['l3_general_organisation_size'].value,
        },
        "employment_type_mix": {
            "description": "Employment Type Mix (percentage)",
            "values": l3_employment_type_mix_values_to_json()
            },
            "position_level_modifiers": {
                "description": "Position Level Modifers",
                "values": l3_position_level_modifiers_values_to_json()
            }
        },
        "org_structure": {}
    }
    
    return output

ws['l3_dataset_details'].children = [
      ws['l3_dataset_details_load_configuration']
    , ws['l3_dataset_details_save_configuration']
    , ws['l3_dataset_details_configuration_name']
    , ws['l3_dataset_details_dataset_configuration_last_modified']
    , ws['l3_dataset_details_dataset_name']
]

In [6]:
# Level 2: Overview
def l2_overview_initialise_values():
    ws['l3_dataset_details_configuration_name'].value                  = config['overview']['dataset_details']['configuration_name']
    ws['l3_dataset_details_dataset_configuration_last_modified'].value = config['overview']['dataset_details']['last_modified']
    ws['l3_dataset_details_dataset_name'].value                        = config['overview']['dataset_details']['dataset_name']


ws['l3_dataset_details_save_configuration'].on_click=(l3_overview_save_configuration)

# Maybe down the track we will want additional information in level 2 overview.
ws['l2_overview'] = ws['l3_dataset_details']

In [7]:
# Level 3: Employee Properties / Organisation Size

def l3_organisation_size_initialise_values():
    ws['l3_general_organisation_size'].value = config['employee_properties']['organisation_size']['value']

In [8]:
# Level 3: Employee Properties / Employment Type Mix
# To Do: Find a better way to address the input values rather than assuming that index 1 of each HBox row will always be the value.
#        Validate totals prior to generating JSON and throw warning if not 100%.

def l3_employment_type_mix_on_value_change(change):
  if change['old'] != change['new']:
    ws['l3_employment_type_mix_total'].value += change['new'] - change['old']

  if ws['l3_employment_type_mix_total'].value == 100:
    ws['l3_employment_type_mix_user_message'].value = 'Employment types total 100% Good to go!'
  elif ws['l3_employment_type_mix_total'].value > 100:
    ws['l3_employment_type_mix_user_message'].value = 'Warning: Employment types total over 100% Please correct mix.'
  else:
    ws['l3_employment_type_mix_user_message'].value = 'Warning: Employment types total under 100% Please correct mix.'


def l3_employment_type_mix_add(additional_mix_details_list = None):
    if additional_mix_details_list == None or additional_mix_details_list.__class__.__name__ == 'Button':
        additional_mix_details_list = [{'mix_name': 'enter employment type', 'mix_percentage': 0}]
    
    additional_mix_list = []

    for index, row in enumerate(additional_mix_details_list):
        swap_widget_id = str(obtain_new_widget_id())
        
        additional_mix_list.append(widgets.HBox(children = [
              widgets.Text(value = row['mix_name'])
            , widgets.BoundedIntText(
                value       = row['mix_percentage'] * 100
              , min         = 0
              , max         = 100
              , step        = 1
              , disabled    = False
            )
            , widgets.Button(
                description = 'Delete'
              , tooltip     = 'delete ' + row['mix_name'] +
                              ' | ID=' + swap_widget_id +
                              ' | Parent=l3_employment_type_mix_body'
            )
        ]))
        
    for row in additional_mix_list:
      row.children[1].observe(handler = l3_employment_type_mix_on_value_change, names = 'value')
      row.children[2].on_click(l3_employment_type_mix_delete)

    ws['l3_employment_type_mix_body'].children += tuple(additional_mix_list)


def l3_employment_type_mix_delete(e):
    for index, row in enumerate(ws['l3_employment_type_mix_body'].children):
        if e.tooltip == row.children[2].tooltip:
            row.children[0].value = '!DELETE ME!'
            row.children[1].value = 0
            l3_employment_type_mix_on_value_change({'old': 0, 'new': 0})
            row.children[0].close()
            row.children[1].close()
            row.children[2].close()
            
            # If we close out the container attempts to add new rows fail with error:
            # AttributeError: 'NoneType' object has no attribute 'comm_id'
            # Appears to be triggered when trying to extend ws['l3_employment_type_mix_body'].children
            # for the time being we'll 0 out the values and leave the containter
            # box in an 'open' state. I'm guessing something tries to iterate
            # over the children list hits something in a closed state and can't
            # handle it.
            # row.close()
            row.layout.visibility = 'hidden'
            break


def l3_employment_type_mix_values_to_json():
    return [
        {'mix_name': row.children[0].value, 'mix_percentage': row.children[1].value / 100}
        for row in ws['l3_employment_type_mix_body'].children
        if row.children[0].value != '!DELETE ME!'
    ]


def l3_employment_type_mix_initialise_values():
    ws['l3_employment_type_mix_body'].children = []
    l3_employment_type_mix_add(config['employee_properties']['employment_type_mix']['values'])

    ws['l3_employment_type_mix_total'].value = sum([
        row.children[1].value
        for row in ws['l3_employment_type_mix_body'].children
    ])
    
    l3_employment_type_mix_on_value_change({'old': 0, 'new': 0})


ws['l3_employment_type_mix_add_type'].on_click(l3_employment_type_mix_add)

ws['l3_employment_type_mix_column_titles'].children = [
      widgets.Text(value = 'Employment Type', disabled = True)
    , widgets.Text(value = 'Percentage'     , disabled = True)
    , widgets.Button(value = ''               , disabled = True, layout = {'visibility': 'hidden'})
]

ws['l3_employment_type_mix_header'].children = [
      ws['l3_employment_type_mix_user_message']
    , ws['l3_employment_type_mix_add_type']
    , ws['l3_employment_type_mix_total']
    , ws['l3_employment_type_mix_column_titles']
]

ws['l3_employment_type_mix'].children = [ws['l3_employment_type_mix_header'], ws['l3_employment_type_mix_body']]

In [12]:
# Level 3: Employee Properties / Position Level Modifiers
# To Do: 

def l3_position_level_modifiers_add(additional_modifier_details_list = None):
    if additional_modifier_details_list == None or additional_modifier_details_list.__class__.__name__ == 'Button':
        additional_modifier_details_list = [{'modifier_name': 'enter position modifier', 'modifier_position': 'pre', 'modifier_level': 0}]
    
    additional_modifier_list = []

    for index, row in enumerate(additional_modifier_details_list):
        swap_widget_id = str(obtain_new_widget_id())
        
        additional_modifier_list.append(widgets.HBox(children = [
              widgets.Text(value = row['modifier_name'])
            , widgets.Dropdown(
                  options     = ['pre', 'post', 'replace']
                , value       = row['modifier_position']
                , description = ''
                , disabled    = False
            )
            , widgets.BoundedIntText(
                value       = row['modifier_level']
              , min         = 1
              , max         = 4
              , step        = 1
              , disabled    = False
            )
            , widgets.Button(
                description = 'Delete'
              , tooltip     = 'delete ' + row['modifier_position'] + 
                              ' | ID=' + swap_widget_id + 
                              ' | Parent=l3_position_level_modifiers_body'
            )
        ]))
        
    for row in additional_modifier_list:
      row.children[3].on_click(l3_position_level_modifiers_delete)

    ws['l3_position_level_modifiers_body'].children += tuple(additional_modifier_list)
    

def l3_position_level_modifiers_delete(e):
    for index, row in enumerate(ws['l3_position_level_modifiers_body'].children):
        if e.tooltip == row.children[3].tooltip:
            row.children[0].value = '!DELETE ME!'
            row.children[0].close()
            row.children[1].close()
            row.children[2].close()
            row.children[3].close()
            
            # If we close out the container attempts to add new rows fail with error:
            # AttributeError: 'NoneType' object has no attribute 'comm_id'
            # Appears to be triggered when trying to extend ws['l3_employment_type_mix_body'].children
            # for the time being we'll 0 out the values and leave the containter
            # box in an 'open' state. I'm guessing something tries to iterate
            # over the children list hits something in a closed state and can't
            # handle it.
            # row.close()
            row.layout.visibility = 'hidden'
            break
    

def l3_position_level_modifiers_values_to_json():
    return [
        {'modifier_name': row.children[0].value, 'modifier_position': row.children[1].value, 'modifier_level': row.children[2].value}
        for row in ws['l3_position_level_modifiers_body'].children
        if row.children[0].value != '!DELETE ME!'
    ]
    

def l3_position_modifiers_initialise_values():
    ws['l3_position_level_modifiers_body'].children = []
    l3_position_level_modifiers_add(config['employee_properties']['position_level_modifiers']['values'])
    

ws['l3_position_level_modifiers_add_modifier'].on_click(l3_position_level_modifiers_add)

ws['l3_position_level_modifiers_column_titles'].children = [
      widgets.Text(value = 'Modifier Name'    , disabled = True)
    , widgets.Text(value = 'Modifier Position', disabled = True)
    , widgets.Text(value = 'Modifier Level'   , disabled = True)
    , widgets.Button(value = ''               , disabled = True, layout = {'visibility': 'hidden'})
]

ws['l3_position_level_modifiers'].children = [
      ws['l3_position_level_modifiers_add_modifier']
    , ws['l3_position_level_modifiers_column_titles']
    , ws['l3_position_level_modifiers_body']
]

In [13]:
# Level 2: Employee Properties
def l2_employee_properties_initialise_values():
    l3_organisation_size_initialise_values()
    l3_employment_type_mix_initialise_values()
    l3_position_modifiers_initialise_values()


ws['l2_employee_properties'].children = [
      ws['l3_general_organisation_size']
    , ws['l3_employment_type_mix']
    , ws['l3_position_level_modifiers']
]

ws['l2_employee_properties'].titles = [
      'Organisation Size'
    , 'Employment Type Mix (percentage)'
    , 'Position Level Modifiers'
]

In [14]:
def initialise_all_values():
    l2_overview_initialise_values()
    l2_employee_properties_initialise_values()

ws['l1_tab'].children = [
      ws['l2_overview']
    , ws['l2_employee_properties']
    , ws['l2_employee_properties']
]

ws['l1_tab'].titles = [
      'Overview'
    , 'Employee Properties'
    , 'Org Structure'
  ]

initialise_all_values()
display(ws['l1_tab'])